In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Comment Score/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Comment Score/test.csv')
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Comment Score/sample_submission.csv')

In [78]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [79]:
train.head()

,UID,comment,date,parent_comment,score
0,Tr-1,NC and NH.,2016-10,"Yeah, I get that argument. At this point, I'd ...",2
1,Tr-2,You do know west teams play against west teams...,2016-11,The blazers and Mavericks (The wests 5 and 6 s...,-4
2,Tr-3,"They were underdogs earlier today, but since G...",2016-09,They're favored to win.,3
3,Tr-4,"This meme isn't funny none of the ""new york ni...",2016-10,deadass don't kill my buzz,-8
4,Tr-5,I could use one of those tools.,2016-12,Yep can confirm I saw the tool they use for th...,6


In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 5 columns):
UID               45000 non-null object
comment           45000 non-null object
date              45000 non-null object
parent_comment    45000 non-null object
score             45000 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [81]:
train.describe()

,score
count,45000.000000
mean,8.133600
std,47.872248
min,-188.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,3192.000000


In [82]:
test.head()

,UID,comment,date,parent_comment
0,Te-1,"Brb, gonna farm my homu",2016-10,So now I KNOW I've sold my soul to the Devil...
1,Te-2,TIL Russia is in Europe,2016-10,The entire CONTINENT of Europe? But arent they...
2,Te-3,Glad they filmed this with an 8mm.,2016-10,Girl drinks whole beer without using her hands
3,Te-4,But what about that cinematic feel?,2016-10,MRW someone defends 24 FPS over 60
4,Te-5,32 spots left as of today.,2016-10,Are there still spots open for this?


In [83]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 4 columns):
UID               30000 non-null object
comment           29998 non-null object
date              30000 non-null object
parent_comment    30000 non-null object
dtypes: object(4)
memory usage: 937.6+ KB


In [0]:
train['text'] = train['comment'] + ' ' + train['parent_comment']
test['text'] = test['comment'] + ' ' + test['parent_comment']

In [0]:
stop = set(stopwords.words('english'))
stop = [token.lower() for token in stop]
train['text'] = train['text'].apply(lambda x: x.replace('   ', ' '))
train['text'] = train['text'].apply(lambda x: x.replace('  ', ' '))
train['text'] = train['text'].apply(lambda x: x.replace('&lt;', ''))
train['text'] = train['text'].apply(lambda x: x.replace('&gt;', ''))
train['text'] = train['text'].apply(lambda x: x.replace('<[^<]+?>', ''))
train['text'] = train['text'].apply(lambda x: x.replace('\r\n', ''))
train['text'] = train['text'].apply(lambda x: x.replace('\r', ''))
train['text'] = train['text'].apply(lambda x: x.replace(r"[^a-zA-Z]+", ''))
train['text'] = train['text'].apply(lambda x: x.replace(('('), ''))
train['text'] = train['text'].apply(lambda x: x.replace((')'), ''))
train['text'] = train['text'].apply(lambda x: x.replace(('{'), ''))
train['text'] = train['text'].apply(lambda x: x.replace(('}'), ''))
train['text'] = train['text'].apply(lambda x: x.replace((','), ''))
train['text'] = train['text'].apply(lambda x: x.replace(('.'), ''))
train['text'] = train['text'].apply(lambda x: x.replace((':'), ''))
train['text'] = train['text'].apply(lambda x: x.replace(('"'), ''))
train['text'] = train['text'].apply(lambda x: x.replace(('?'), ''))
train['text'] = train['text'].apply(lambda x: re.sub(r'\b\d+\b', '', x))
train['text'] = train['text'].apply(lambda x: ' '.join([item.lower() for item in x.split() if item.lower() not in stop]))
train['text'] = train['text'].apply(lambda x: ' '.join([item for item in x.split() if len(item)>2]))

In [0]:
test['text'] = test['text'].astype(str).apply(lambda x: x.replace('   ', ' '))
test['text'] = test['text'].apply(lambda x: x.replace('  ', ' '))
test['text'] = test['text'].apply(lambda x: x.replace('&lt;', ''))
test['text'] = test['text'].apply(lambda x: x.replace('&gt;', ''))
test['text'] = test['text'].apply(lambda x: x.replace('<[^<]+?>', ''))
test['text'] = test['text'].apply(lambda x: x.replace('\r\n', ''))
test['text'] = test['text'].apply(lambda x: x.replace('\r', ''))
test['text'] = test['text'].apply(lambda x: x.replace(r"[^a-zA-Z]+", ''))
test['text'] = test['text'].apply(lambda x: x.replace(('('), ''))
test['text'] = test['text'].apply(lambda x: x.replace((')'), ''))
test['text'] = test['text'].apply(lambda x: x.replace(('{'), ''))
test['text'] = test['text'].apply(lambda x: x.replace(('}'), ''))
test['text'] = test['text'].apply(lambda x: x.replace((','), ''))
test['text'] = test['text'].apply(lambda x: x.replace(('.'), ''))
test['text'] = test['text'].apply(lambda x: x.replace((':'), ''))
test['text'] = test['text'].apply(lambda x: x.replace(('"'), ''))
test['text'] = test['text'].apply(lambda x: x.replace(('?'), ''))
test['text'] = test['text'].apply(lambda x: re.sub(r'\b\d+\b', '', x))
test['text'] = test['text'].apply(lambda x: ' '.join([item.lower() for item in x.split() if item.lower() not in stop]))
test['text'] = test['text'].apply(lambda x: ' '.join([item for item in x.split() if len(item)>2]))

In [87]:
test['text'][:5]

0        brb gonna farm homu know i've sold soul devil
1    til russia europe entire continent europe aren...
2    glad filmed 8mm girl drinks whole beer without...
3               cinematic feel mrw someone defends fps
4                    spots left today still spots open
Name: text, dtype: object

In [0]:
vector = CountVectorizer(ngram_range=(1,4), max_features=35)

X = vector.fit_transform(train['text'])
Xtest = vector.transform(test['text'])
y = train['score']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=0, n_estimators=70, max_depth=2)
# model = MultinomialNB(alpha=1)
# model = LinearSVR(random_state=42, loss='l1', C=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('r2 score:', r2_score(y_test, y_pred))
print('rmse', np.sqrt(mean_squared_error(y_test, y_pred)))


r2 score: -0.0013592279444660793
rmse 47.47666489221209


In [0]:
model.fit(X, y)
yout = model.predict(Xtest)

In [72]:
submission.head()

,UID,score
0,Te-1,5.973084
1,Te-2,15.330436
2,Te-3,5.973084
3,Te-4,5.973084
4,Te-5,5.669788


In [0]:
df = pd.DataFrame()
df['UID'] = test['UID']
df['score'] = yout
df.to_csv('/content/drive/My Drive/Colab Notebooks/Comment Score/submit.csv', index=False)